In [ ]:
import importlib
%load_ext autoreload
%autoreload 2
from src.pyAOM_utils import *
import re
import statsmodels.api as sm
from scipy.optimize import curve_fit
import shutil

In [ ]:
# heterocycles
# define the directory with single molecules
single_molecules_dir='heterocycles'
# label
dataset='heterocycles'
# basis set
basis='DZVP-GTH'
# XC
xc='PBE'
# single molecule suffix
suffix='.xyz'
# list of molecules in the dataset
single_molecules=[i.strip('.xyz') for i in os.listdir('heterocycles')]

In [ ]:
# the HAB11+ database (except benzene and acetylene)
# define the directory with single molecules
single_molecules_dir='single_molecules'
# label
dataset='HAB11+'
# basis set
basis='DZVP-GTH'
# XC
xc='PBE'
# single molecule suffix
suffix='.xyz'
# list of molecules in the dataset
single_molecules=[
    'ethylene',
    'cyclopropene',
    'cyclobutadiene',
    'cyclopentadiene',
    'furane',
    'pyrrole',
    'thiophene',
    'imidazole',
    'phenol']

In [ ]:
# the HAB7- database
# define the directory with single molecules
single_molecules_dir='single_molecules'
# label
dataset='HAB7-'
# basis set
basis='DZVP-GTH'
# XC
xc='PBE'
# single molecule suffix
suffix='.xyz'
# list of molecules in the dataset
single_molecules=[
    '3cene',
    '4cene',
    '5cene',
    'anthraceneF',
    'perylene',
    'perylene_diimide',
    'porph']

In [ ]:
# generate cp2k:qs input files
if os.path.exists('output')==False:
    os.mkdir('output')
directory=f'output/{dataset}_{basis}_{xc}'
if os.path.exists(directory)==False:
    os.mkdir(directory)
for i in single_molecules:
    shutil.copyfile(f'{single_molecules_dir}/{i}{suffix}',f'{directory}/{i}{suffix}')
    single=System(f'{directory}/{i}{suffix}')
    single.prep_cp2k_single(basis=basis)

In [ ]:
# optional
# prepare single cp2k input files and submission script entries for ARCHER
# for different HPC infrastructure, just change the print statement accordingly
if os.path.exists(f'{directory}/archer')==False:
    os.mkdir(f'{directory}/archer')
output_list=[]
for mol in single_molecules:
    for i in os.listdir(f'{directory}/{mol}'):
        if i.endswith('.inp')==True:
            shutil.copyfile(f'{directory}/{mol}/{i}',f'{directory}/archer/{mol}_{i}')
            fp=open(f'{directory}/archer/{mol}_{i}',mode='r')
            inp=fp.readlines()
            fp.close()
            for c,j in enumerate(inp):
                if j.find(f'@INCLUDE {i.split(".inp")[0]}_subsys.include')!=-1:
                    inp[c]=f'@INCLUDE {mol}_{i.split(".inp")[0]}_subsys.include\n'
                if j.find(f'PROJECT_NAME {i.split(".inp")[0]}')!=-1:
                    inp[c]=f'PROJECT_NAME {mol}_{i.split(".inp")[0]}\n'
            with open(f'{directory}/archer/{mol}_{i}',mode='w') as fp:
                for j in inp:
                    print(j,file=fp,end='')
            out=f'{mol}_{i.split(".inp")[0]}.out'
            output_list.append([out,mol,i])
            print(f'aprun -n 192 cp2k.popt -i {mol}_{i} > {out}')
        if i.endswith('.include')==True:
            shutil.copyfile(f'{directory}/{mol}/{i}',f'{directory}/archer/{mol}_{i}')

In [ ]:
# optional
# provided that all output files are in 'archer', distribute them to all output directories
for i in output_list:
    shutil.copyfile(f'{directory}/archer/{i[0]}',f'{directory}/{i[1]}/{i[2].split(".inp")[0]}.out')

In [ ]:
# optional
# back-compatibility check with C implementation
# prepare separate MOLog files
for i in output_list:
    filename=f'{directory}/archer/{i[0]}'
    fp=open(filename,mode='r')
    out=fp.readlines()
    fp.close()
    bounds=[]
    for c,line in enumerate(out):
        if line.find('ALPHA')!=-1:
            bounds.append(c-2)
        if line.find('HOMO-LUMO')!=-1:
            bounds.append(c+2)
    with open(f'{filename.split(".out")[0]}.MOLog',mode='w') as fp:
        for j in range(bounds[0],bounds[2]):
            print(out[j],file=fp,end='')

In [ ]:
# STO parameters for GTO-STO projection step
STO_proj_dict={
        'H':{'STOs':1  ,'1s':1.0000},
        'C':{'STOs':1+3,'2s':1.6083,'2p':1.4440},
        'N':{'STOs':1+3,'2s':1.9237,'2p':1.3340},
        'O':{'STOs':1+3,'2s':2.2458,'2p':2.2266},
        'F':{'STOs':1+3,'2s':2.5638,'2p':2.5500},
        'S':{'STOs':1+3,'3s':2.1223,'3p':1.5850},
    }

In [ ]:
# Provided that we retrieved the .out cp2k output files from the single molecule
# DFT runs, carry out the GTO-STO projections, save orbital completeness values, 
# and store AOM pi coefficients in appropriate files
# The mol_type_driver dictiorary is prepared as to include HOMO and LUMO only!
proj_results={}
for i in single_molecules:
    # configuration dict
    proj_driver={
        'xyz':f'{directory}/{i}.xyz',
        'basis':basis,
        'out_dir':f'{directory}/{i}',
        'cp2k_basis_file':'cp2k_files/GTH_BASIS_SETS'
    }
    # read system
    single=System(proj_driver['xyz'])
    # resolve auxiliary dict with input parameters 
    mol_type_driver=single.resolve_mol_type_driver(proj_driver)
    print(mol_type_driver)
    # carry out STO projection
    single.apply_STO_projection(STO_proj_dict,proj_driver,False)
    # apply AOM info to all molecules
    single.apply_AOM_types()
    # save to file(s)
    single.store_AOM_types(proj_driver)
    # store orbital compl.
    proj_results[i]={'orb_compl':single.molecule[0].orb_compl_dict}

In [ ]:
# Inspect HOMO/LUMO orbital completeness
HOMO_dict={}
LUMO_dict={}
for key,value in proj_results.items():
    for c,i in enumerate(value['orb_compl'].keys()):
        if c==0:
            HOMO_dict[key]=i
        else:
            LUMO_dict[key]=i
import matplotlib.pyplot as plt
import seaborn as sns
orb_compl_HOMO=[]
orb_compl_LUMO=[]
for key,value in proj_results.items():
    for i in value.values():
        for c,j in enumerate(i.values()):
            if c==0:
                orb_compl_HOMO.append(j)
            else:
                orb_compl_LUMO.append(j)
sns.set(color_codes=True)
sns.set_context("talk")
plt.figure()
sns.distplot(orb_compl_HOMO,rug=True)
plt.figure()
sns.distplot(orb_compl_LUMO,rug=True);

In [ ]:
# print values
for key,value in proj_results.items():
    compl=[]
    for i in value['orb_compl'].values():
        compl.append(i)
    print(f'{key}\t{compl}')

In [ ]:
# Read ab-initio CTI reference values
# fp=open('couplings/abinitio_HAB11+.dat',mode='r')
fp=open('couplings/abinitio_HAB7-.dat',mode='r')
data=fp.readlines()
fp.close()
data=[re.split('\t|_',i.strip()) for i in data]
# for molecule names with underscores
for i in range(len(data)):
    if len(data[i])>3:
        name=''
        for j in range(len(data[i])-2):
            name+=data[i][j]+'_'
        data[i]=[name.strip('_'),data[i][-2],data[i][-1]]
abinitio_ref={}
for i in data:
    abinitio_ref[i[0]]={}
for i in data:
    abinitio_ref[i[0]].update({i[1]:float(i[-1])})

In [ ]:
# Define STO exponents
AOM_overlap_dict={
        'H':{'STOs':1  ,'1s':1.0000},
        'C':{'STOs':1+3,'2s':1.6083,'2p':1.0300},
        'N':{'STOs':1+3,'2s':1.9237,'2p':1.2050},
        'O':{'STOs':1+3,'2s':2.2458,'2p':2.2266},
        'F':{'STOs':1+3,'2s':2.5638,'2p':2.5500},
        'S':{'STOs':1+3,'3s':2.1223,'3p':1.5850},
    }

In [ ]:
# Calculate dimer AOM overlaps
# make sure to select the appropriate MO dictionary!
AOM_Sab={i:{} for i in single_molecules}
dist=[3.5,4.0,4.5,5.0]
for i in single_molecules:
    for j in dist:
        xyz=f'dimers/dimer_{i}_{j}A.xyz'
        out_dir=f'{directory}/{i}'
#         MO=HOMO_dict[i]
        MO=LUMO_dict[i]
        aom_driver={
            'xyz':xyz,
            'out_dir':out_dir,
        }
        # read system
        dimer=System(aom_driver['xyz'])
        # load AOM dict from file
        for k,l in enumerate(dimer.mol_type):
            dimer.molecule[k].read_AOM_from_file(aom_driver,l)
        # calculate AOM overlap
        _,_,_,_,Sab=dimer.calculate_AOM_overlap(AOM_overlap_dict,MO,MO)
        AOM_Sab[i].update({f'{j}A':Sab})

In [ ]:
# inspect values
AOM_Sab

In [ ]:
# linear fit
AOM_values=[abs(i) for key,value in AOM_Sab.items() for i in value.values()]
ref_values=[i for key,value in abinitio_ref.items() for i in value.values()]
# regression on linear data
X=AOM_values
Y=ref_values
model = sm.OLS(Y, X).fit()
print(model.summary())
xline=[min(AOM_values),max(AOM_values)]
yline=[model.params[0]*i for i in xline]
plt.plot(AOM_values,ref_values,'.')
plt.plot(xline,yline);

In [ ]:
# regression on logarithmic data
AOM_values_log=[math.log(i,10) for i in AOM_values]
ref_values_log=[math.log(i,10) for i in ref_values]
def f(x,b):
    return x+b
popt, pcov = curve_fit(f, AOM_values_log, ref_values_log)
xline=[min(AOM_values_log),max(AOM_values_log)]
yline=[i+popt for i in xline]
plt.plot(AOM_values_log,ref_values_log,'.')
plt.plot(xline,yline);
print(10**popt)
print(pcov)